In [ ]:
import msmrd2
import numpy as np

In [ ]:
# Particle definition
D = 1
Drot = 0.05
bodytype = 'rod'
position = np.array([1,1,1])
orientation = np.array([1,0,0,0])
part1 = msmrd2.particle(D, Drot, bodytype, position, orientation)
part1.setOrientVector(np.array([1.,0.,0.]))

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.001
seed = 0 # Seed = -1 used random device as seed
rotation = True
intg = msmrd2.integrators.odLangevin(dt, seed, rotation)

In [ ]:
# Define and set potential
scalefactor = 100.0
Efieldvector = np.array([0,0,1])
dipolePot = msmrd2.potentials.dipole(scalefactor, Efieldvector)
intg.setExternalRodPotential(dipolePot)
#gauss3Dpot = msmrd2.potentials.gaussians3D(15,3,20,-1)
#intg.setExternalPotential(gauss3Dpot)

In [ ]:
# Integrate only one particle and print position and orientation
t = 0
timesteps = 5000
print('{:<10s}{:<10s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Orientation 1"))
for i in range(timesteps):
    print('{:<10d}{:<10f}{:<40s}{:<40s}'.format(i, intg.clock, str(part1.position), str(part1.orientvector)))
    intg.integrate(part1)